In [1]:
import json
import schluesselregex as rex
import re
import numpy as np
from geopy.geocoders import Nominatim

In [2]:
with open('hida\HidaDict.json') as f:
  hidaDict = json.load(f)

with open('hida\HidaDictPLZ.json') as f:
  hidaDictPLZ = json.load(f)

with open('pdf\DenkmallisteBerlin.json') as g:
  pdfDict = json.load(g)

with open('hida\entryKeysHida.json') as h:
  entryHida = json.load(h)

In [3]:
#helper functions
def remElementDictionary(dict,listElements):
    for element in listElements:
        if element in dict.keys():
            del dict[element]
    return dict

def hidaKeyListConverter(listElements):
    newList=[]
    for element in listElements:
        newList.append(entryHida[element])
    return newList

In [4]:
#stelt die richtige Beziehung für T her
objNrKeys=list(hidaDict.keys())
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if ('Beziehung' in hidaDict[key].keys()):
        hidaDict[key]['Beziehung2']={}
        for bezKey in hidaDict[key]['Beziehung'].keys():# there are multiple bezKey, bez= Beziehung
            if bezKey not in hidaDict.keys():#Beziehung e.g. do not explicitely refer to the 09096451,T and 9075895,T  but instead to 09096451 and 9075895
                bezKeyT = bezKey+',T'
            else:
                bezKeyT = bezKey
            hidaDict[key]['Beziehung2'][bezKeyT]=hidaDict[key]['Beziehung'][bezKey]
        del hidaDict[key]['Beziehung']
        hidaDict[key]['Beziehung']= hidaDict[key]['Beziehung2']
        del hidaDict[key]['Beziehung2']
        #print(hidaDict[key]['Beziehung'])

In [5]:
#doppelte Adressen
# adresse={}
# objNrKeys=list(hidaDict.keys())
# for n in range(0,len(objNrKeys)):
#     key=objNrKeys[n]
#     if ('Listentext' in hidaDict[key].keys()):
#         adresse[key]=[]
#         content=hidaDict[key]['Listentext']
#         #print('content: ', content)
#         adr=set(rex.getRegex(content).adresse)
#         #print('adr: ', adr)
#         for elAdr in adr:
#             adresse.keys()[adresse.values().index(elAdr)]
#             #newElAdr=re.findall('\D+',elAdr)[0]
#             if elAdr not in adresse.values():
#                 adresse[key].append(elAdr)
#             else:
                
                
#             elif hidaDict[key]['Bez-OBJ-Nr']!=key:
#                 print('objNr: ',key,'Bez-OBJ-Nr: ',hidaDict[key]['Bez-OBJ-Nr'],'elAdr: ', elAdr)
# print('adresse: ', adresse)
        


In [6]:
def removeR2(s):
    for el in re.findall(r'\(.*?\)', s):
        if 'siehe' in el:
            s=s.replace(el,"")
    return s

def findR2(s):
    elList=[]
    for el in re.findall(r'\(.*?\)', s):
        if 'siehe' in el:
            elList.append(el)
    return elList

In [7]:
adresse=[]
keyAdresse=[]
bezKeyAdresse=[]
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    #if ('Bez-OBJ-Nr' in hidaDict[key].keys()) and ('Denkmalart' not in hidaDict[key].keys()):
    #if 'Bez-OBJ-Nr' in hidaDict[key].keys() and len(hidaDict[key]['Bez-OBJ-Nr'])!= len(hidaDict[key]['Beziehung']):
    #if 'Bez-OBJ-Nr' in hidaDict[key].keys() and "Teil von" in hidaDict[key]['Beziehung'] and 'Denkmalart' in hidaDict[key].keys() and hidaDict[key]['Denkmalart']=='Gesamtanlage':
    #if 'Beziehung' in hidaDict[key].keys() and 'Denkmalart' in hidaDict[key].keys() and hidaDict[key]['Denkmalart']=='Gesamtanlage':
    #if 'Denkmalart' in hidaDict[bezKey+ ' '].keys() and hidaDict[bezKey+' ']['Denkmalart']=='Ensemble':
    #if bezKey+' ' in hidaDict.keys() and not bezKey+',T' in hidaDict.keys():
    #if 'Straße' not in hidaDict[key].keys() and 'T,0' not in key:
        #i+=1
        #print('i=: ', i ,'key: ', key)
        #print(hidaDict[key]['Listentext'],'\n')
        #print('bezKey: ', hidaDict[key]['Beziehung'])

In [8]:
#finde heraus welche Objekte mehrere Bezirke enthalten (außer 'T,0'):
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Bezirk' in hidaDict[key].keys() and 'T,0' not in key and '&' in hidaDict[key]['Bezirk']:
        print('i=: ', i ,'key: ', key,hidaDict[key]['Bezirk'])
        i+=1

i=:  0 key:  09097763 Friedrichshain-Kreuzberg & Mitte & Pankow & Treptow-Köpenick
i=:  1 key:  09080614,T Friedrichshain-Kreuzberg & Treptow-Köpenick
i=:  2 key:  09075292 Steglitz-Zehlendorf & Charlottenburg-Wilmersdorf
i=:  3 key:  09065204 Mitte & Pankow
i=:  4 key:  09055092,T Tempelhof-Schöneberg & Neukölln
i=:  5 key:  09055090 Tempelhof-Schöneberg & Neukölln
i=:  6 key:  09055089 Friedrichshain-Kreuzberg & Tempelhof-Schöneberg
i=:  7 key:  09050364,T Charlottenburg-Wilmersdorf & Friedrichshain-Kreuzberg & Mitte & Neukölln
i=:  8 key:  09045000 Reinickendorf & Pankow
i=:  9 key:  09040492 Charlottenburg-Wilmersdorf & Spandau
i=:  10 key:  09040456 Charlottenburg-Wilmersdorf & Friedrichshain-Kreuzberg & Tempelhof-Schöneberg
i=:  11 key:  09030358 Mitte & Pankow
i=:  12 key:  09011323,T Charlottenburg-Wilmersdorf & Mitte & Friedrichshain-Kreuzberg


In [9]:
#finde heraus welche Objekte kein Ortsteil enthalten (außer 'T,0'):
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Ortsteil' not in hidaDict[key].keys() and 'T,0' not in key:
        print('i=: ', i ,'key: ', key,hidaDict[key]['Bezirk'])
        i+=1

i=:  0 key:  09097763 Friedrichshain-Kreuzberg & Mitte & Pankow & Treptow-Köpenick


In [10]:
#finde ob Sachbegriffe enthalten sind (außer 'T,0'):
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Sachbegriff' not in hidaDict[key].keys() and 'T,0' not in key:
        print('i=: ', i ,'key: ', key)
        i+=1

In [11]:
#finde heraus welche Objekte keine Adresse enthalten (außer 'T,0'):
objNoAdress=[]
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Straße' not in hidaDict[key].keys() and 'T,0' not in key:
        print('i=: ', i ,'key: ', key,hidaDict[key]['Denkmalname'])
        objNoAdress.append(key)
        i+=1

i=:  0 key:  09097763 ['Berliner Mauer']
i=:  1 key:  09096022 ['Stellwerk Rs VI']
i=:  2 key:  09096020 ['Ostbahnüberführung']
i=:  3 key:  09095359 ['Mechanisches Stellwerk']
i=:  4 key:  09095356 ['Wasserturm']
i=:  5 key:  09095209 ['Lokomotiv - Rundschuppen']
i=:  6 key:  09095001 ['Ringbahnbrücke Oberspree, Stellwerk und Unterführung Alt-Stralau mit Bahndamm-Mauer']
i=:  7 key:  09085839 ['Westlicher Abzugsgraben']
i=:  8 key:  09085803 ['Siemensbahn (zwischen Bezirksgrenze und Bahnhof Gartenfeld) ', ' Bahnhof Wernerwerk ', ' Bahnhof Siemensstadt ', ' Endbahnhof Gartenfeld ', ' Stellwerk Gartenfeld ', ' Viadukt Rohrdamm, Gleichrichterwerk Rohrdamm']
i=:  9 key:  09085704 ['östlicher Abzugsgraben']
i=:  10 key:  09075622 ['Einschnittbahn der U-Bahnlinie U 1 zwischen U-Bhf. Podbielskiallee und U-Bhf. Krumme Lanke mit Straßenbrücken über Clayallee, Landoltweg, Habelschwerdter Allee, Fabeckstraße, Königin-Luise-Straße']
i=:  11 key:  09075538 ['Glienicker Brücke']
i=:  12 key:  09055

In [12]:
#finde heraus welche Objekte auf sich selbst referenzieren
objNrKeys=list(hidaDict.keys())
error=[]
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Beziehung' in hidaDict[key].keys():
        for bezKey in hidaDict[key]['Beziehung'].keys():
            if bezKey == key:#  and 'Denkmalart' in hidaDict[key].keys() and hidaDict[key]['Denkmalart']=='Ensembleteil':
                bezDenkmalArt=hidaDict[bezKey]['Denkmalart']
                denkmalArt=hidaDict[key]['Denkmalart']
                i+=1
                error.append(key)
                print('i: ',i,' key: ',key,' bezKey: ',bezKey, ' wobei ',denkmalArt,hidaDict[key]['Beziehung'][bezKey], bezDenkmalArt,'Adresse: ',hidaDict[key]['Adresse'])

In [13]:
# finde, ob außer Ensembles auch andere Arten eine Teil-Beziehung aufweisen
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Beziehung' in hidaDict[key].keys():
        for bezKey in hidaDict[key]['Beziehung'].keys():
            if ("mjsv") not in hidaDict[key]['Beziehung'][bezKey] and 'Denkmalart' in hidaDict[key].keys() and hidaDict[key]['Denkmalart']=='Ensembleteil':
                if bezKey not in hidaDict.keys():#Beziehung e.g. do not explicitely refer to the 09096451,T and 9075895,T  but instead to 09096451 and 9075895
                    bezKeyT = bezKey+',T'
                    bezDenkmalArt=hidaDict[bezKeyT]['Denkmalart']
                else:
                    bezKeyT = bezKey
                    bezDenkmalArt=hidaDict[bezKeyT]['Denkmalart']
                denkmalArt=hidaDict[key]['Denkmalart']
                if bezDenkmalArt!='Ensemble':
                    i+=1
                    print('i: ',i,' key: ',key,' bezKey: ',bezKeyT, ' wobei ',denkmalArt,hidaDict[key]['Beziehung'][bezKey], bezDenkmalArt)

i:  1  key:  09075583  bezKey:  09045986  wobei  Ensembleteil siehe auch Gartendenkmal Gartendenkmal
i:  2  key:  09035427  bezKey:  09020495  wobei  Ensembleteil siehe auch Baudenkmal Baudenkmal
i:  3  key:  09020538  bezKey:  09046327  wobei  Ensembleteil siehe auch Gartendenkmal Gartendenkmal


In [14]:
# finde, ob es Beziehungen von einem Objekt zu einem anderen existiert. Hier sollen die Beziehungen, die keine Adressen enthalten vernachlässigt werden.
objNrKeys=list(hidaDict.keys())
i=0
exception=[]
#finde objnr die keine Adresse enthalten
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Straße' not in hidaDict[key].keys() and 'T,0' not in key:
        exception.append(key)
        #print('key: ', key , 'Denkmalname: ', hidaDict[key]['Denkmalname'])
#print(exception)
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Beziehung' in hidaDict[key].keys():
        for bezKey in hidaDict[key]['Beziehung'].keys():
            if ("Teil" or "siehe") in hidaDict[key]['Beziehung'][bezKey] and 'Denkmalart' in hidaDict[key].keys() and hidaDict[key]['Denkmalart']!='Ensembleteil':
                bezDenkmalArt=hidaDict[bezKeyT]['Denkmalart']
                denkmalArt=hidaDict[key]['Denkmalart']
                if bezDenkmalArt=='Ensembleteil' and bezKey not in exception and key not in exception:
                    i+=1
                    bezBezKey=list(hidaDict[bezKey]['Beziehung'].keys())[0]
                    bezBezDenkmarArt=hidaDict[bezBezKey]['Denkmalart']
                    print('i: ',i,' key: ',key,' bezKey: ',bezKey, ' wobei ',denkmalArt,hidaDict[key]['Beziehung'][bezKey], bezDenkmalArt,denkmalArt,denkmalArt,'bezBezKey: ',bezBezKey,'bezBezDenkmarArt: ',bezBezDenkmarArt)

In [15]:
# finde, ob ein Ensemble auf andere mit siehe oder Teil von verweist
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'Beziehung' in hidaDict[key].keys():
        for bezKey in hidaDict[key]['Beziehung'].keys():
            if ("siehe" or "Teil") in hidaDict[key]['Beziehung'][bezKey] and 'Denkmalart' in hidaDict[key].keys() and hidaDict[key]['Denkmalart']=='Ensemble':
                i+=1
                print('i=: ', i ,'key: ', key,'bezKey: ', hidaDict[key]['Beziehung'])

i=:  1 key:  09090204 bezKey:  {'09090205': 'siehe auch Gesamtanlage'}
i=:  2 key:  09085334 bezKey:  {'09046059': 'siehe auch Gartendenkmal', '09085055': 'siehe auch Gartendenkmal', '09085369,T': 'siehe auch Gesamtanlage'}
i=:  3 key:  09085334 bezKey:  {'09046059': 'siehe auch Gartendenkmal', '09085055': 'siehe auch Gartendenkmal', '09085369,T': 'siehe auch Gesamtanlage'}
i=:  4 key:  09085334 bezKey:  {'09046059': 'siehe auch Gartendenkmal', '09085055': 'siehe auch Gartendenkmal', '09085369,T': 'siehe auch Gesamtanlage'}
i=:  5 key:  09020790 bezKey:  {'09046359,T': 'siehe auch Gartendenkmal'}


In [16]:
#überprüfe mit den seprarierten bez-Keys, ob fehlende ObjNr vorhanden sind. Danach werden diese aus den Eckdaten manuell entfernt
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    bezObj={}
    if 'Bez-OBJ-Nr' in hidaDict[key].keys():
        bezKeys=hidaDict[key]['Bez-OBJ-Nr']
        bezValue=hidaDict[key]['Beziehung']
        for i in range(len(bezKeys)):
            if '&' in bezKeys[i]:
                find = re.findall(r'\d{8}', bezKeys[i])
                for elBezKey in find:
                    bezObj[elBezKey]=bezValue[i]
            else:bezObj[bezKeys[i]]=bezValue[i]
        for bezKey in bezObj.keys():
            if bezKey+' ' not in hidaDict.keys() and bezKey+',T ' not in hidaDict.keys():
                i+=1
                print('i=: ', i ,'key: ', key, 'bezKey: ',bezKey)
                print('bezKey: ', hidaDict[key]['Beziehung'], hidaDict[key]['Bez-OBJ-Nr'])

In [17]:
adresse=[]
keyAdresse=[]
bezKeyAdresse=[]
objNrKeys=list(hidaDict.keys())
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    #09097763 Berliner Mauer zieht sich über mehrere Bezirke und hat besitzt keinen bestimmten Ortteil
    #alle Objr die referenziert sind, existieren nun auch wieder
    #alle ensembleteile sind ein Teil eines Ensembles. 09020538  bezKey:  09046327, 09035427  bezKey:  09020495, 09075583  bezKey:  09045986 sind zusätzlich auf einem Gartendenkmal bzw. Baudenkmal verwiesen ("siehe")
    #alle Objekte (außer "T,0..."") enthalte den Eintrag Bezirk
    #ensemble ist kein Teil von etwas anderem aber es gibt siehe Verweise bei folgenden Ensembles: 09090204,09085334,09020790
    #ensemble steht nicht in direkter Beziehung (grad 1) zu einem anderen Ensemble
    #alle Objekte haben den Eintrag Denkmalart, bis auf die Teilobjekte 9075895,T001 etc. Dieser erhalten ihre Denkmalart vom Hauptobjekt 9075895,T
    #ist Bez-OBJ und Beziehung im Objekt und ist Objekt kein Ensemble, taucht im Listentext (Beziehung + Adresse von Bez-OBJ auf). Ist Objekt aber ein Ensemble dann taucht eine Zeile mit Beziehung: Adresse von Bez-OBJ
    if 'PLZ' in hidaDictPLZ[key]:
        hidaDict[key]['PLZ']=hidaDictPLZ[key]['PLZ']
    if ('Beziehung' in hidaDict[key].keys()):
        if 'T' in key:#the TeilObjNr 09096451,T,001 09096451,T00... do not contain the Key Denkmalart. However, instead the mainObj does contain is, i.e.  09096451,T
            DenkmalArt=hidaDict[key[:10]]['Denkmalart']
        else:
            denkmalArt=hidaDict[key]['Denkmalart']
        for bezKey in hidaDict[key]['Beziehung'].keys():# there are multiple bezKey, bez= Beziehung
            bezDenkmalArt=hidaDict[bezKey]['Denkmalart']
            bezArt = hidaDict[key]['Beziehung'][bezKey]
            if 'Teil' in bezArt:
                #Haupt-Main
                hidaDict[bezKey].setdefault('Status',{})['Haupt']=True
                if denkmalArt== 'Ensembleteil' and bezDenkmalArt== 'Ensemble':
                    hidaDict[bezKey].setdefault('Ensemble-Teil',[]).append(key)
                hidaDict[bezKey].setdefault('Teil-Beziehung',{})[key]=denkmalArt
                hidaDict[bezKey].setdefault('LinkGrad1ObjNr',[]).append(key)
                #Teil-Sub
                hidaDict[key].setdefault('Status',{})['Teil']=True
                hidaDict[key].setdefault('HauptObjNr',{})[bezKey]=bezDenkmalArt
                hidaDict[key].setdefault('LinkGrad1ObjNr',[]).append(bezKey)
            elif 'siehe' in bezArt:
                hidaDict[bezKey].setdefault('Status',{})['Siehe']=True#gilt nur wenn Siehe Beziehungen, symmetrische Beziehungen sind
                hidaDict[bezKey].setdefault('Siehe-Beziehung',{})[key]=denkmalArt#gilt nur wenn Siehe Beziehungen, symmetrische Beziehungen sind
                hidaDict[bezKey].setdefault('LinkGrad1ObjNr',[]).append(key)
                hidaDict[key].setdefault('Status',{})['Siehe']=True#gilt nur wenn Siehe Beziehungen, symmetrische Beziehungen sind
                hidaDict[key].setdefault('Siehe-Beziehung',{})[bezKey]=bezDenkmalArt#gilt nur wenn Siehe Beziehungen, symmetrische Beziehungen sind
                hidaDict[key].setdefault('LinkGrad1ObjNr',[]).append(bezKey)
            else:
                print(key,bezKey)
                print('Weder Teil noch siehe in Beziehung enthalten')
            if ('Beziehung' in hidaDict[bezKey].keys()):
                for bezBezKey in hidaDict[bezKey]['Beziehung'].keys():#Beziehunge 2.Grades, there are multiple bezKey, bez= Beziehung
                    if bezBezKey != key and bezBezKey not in hidaDict[key]['LinkGrad1ObjNr']:
                        hidaDict[key].setdefault('LinkGrad2ObjNr',[]).append(bezBezKey)
                        hidaDict[bezBezKey].setdefault('LinkGrad2ObjNr',[]).append(key)
                    
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if 'LinkGrad1ObjNr' in hidaDict[key].keys():
        hidaDict[key]['LinkGrad1ObjNr']=list(set(hidaDict[key]['LinkGrad1ObjNr']))
    if 'LinkGrad2ObjNr' in hidaDict[key].keys():
        hidaDict[key]['LinkGrad2ObjNr']=list(set(hidaDict[key]['LinkGrad2ObjNr']))
#todo:Dann auch die die keine Beziehung haben anschauen. Aber erst nach Beziehung identifikation aus Listentext?

In [18]:
objNrKeys=list(hidaDict.keys())
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    if ('LinkGrad2ObjNr' in hidaDict[key].keys() and 'Teil-Beziehung'in hidaDict[key].keys() and 'Siehe-Beziehung'in hidaDict[key].keys()):
        print(key)

09085037
09046414
09040273,T
09040272,T
09040271,T
09040270,T
09011333


In [19]:
delKey=['Beziehung','Straße','Hausnummer','Adresse']
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    for el in delKey:
        if el in hidaDict[key].keys():
            del hidaDict[key][el]
print(delKey,' has been deleted')
#save HidaDict as json file
with open('hida\ExtendedHidaDict.json', 'w') as fp:
    json.dump(hidaDict, fp,  indent=4,ensure_ascii=False)
print('successfully saved')

['Beziehung', 'Straße', 'Hausnummer', 'Adresse']  has been deleted
successfully saved


In [20]:
#Schau, ob es Dublikate bzgl der Adressen existieren
#Ohne Höhrere Beziehungsgrade existieren 740 Dublikate
#ohne Ortsteil existieren 37 Adressen, die als Dublikate erscheinen:
#['Berliner Straße 24', 'Berliner Straße 25', 'Dorfstraße 10', 'Dorfstraße 12', 'Dorfstraße 13', 'Dorfstraße 17', 'Dorfstraße 18', 'Dorfstraße 19', 'Dorfstraße 2', 'Dorfstraße 21', 'Dorfstraße 23', 'Dorfstraße 28', 'Dorfstraße 3', 'Dorfstraße 30', 'Dorfstraße 31', 'Dorfstraße 32', 'Dorfstraße 7', 'Dorfstraße 8', 'Dorfstraße 9', 'Gartenstraße 1', 'Grünstraße 25', 'Hanseatenweg 10', 'Hauptstraße 12', 'Hauptstraße 13', 'Hauptstraße 15', 'Hauptstraße 17', 'Hauptstraße 23', 'Hauptstraße 26', 'Hauptstraße 28', 'Hauptstraße 29', 'Hauptstraße 30', 'Hauptstraße 41', 'Hauptstraße 42', 'Hauptstraße 45', 'Hauptstraße 53', 'Hauptstraße 55', 'Hauptstraße 58']
#ohne Bezirk und Ortsteil existieren 118 Adressen, die als Dublikate erscheinen:
#['Albrechtstraße 12', 'Albrechtstraße 13', 'Albrechtstraße 5', 'Albrechtstraße 7', 'Albrechtstraße 8', 'Auguststraße 1', 'Auguststraße 18', 'Auguststraße 19', 'Auguststraße 2', 'Auguststraße 8', 'Bahnhofstraße 30', 'Bahnhofstraße 31', 'Berliner Straße 24', 'Berliner Straße 25', 'Berliner Straße 26', 'Breite Straße 35', 'Breite Straße 43', 'Breite Straße 44', 'Breite Straße 45', 'Breite Straße 46', 'Dorfstraße 1', 'Dorfstraße 10', 'Dorfstraße 10A', 'Dorfstraße 11', 'Dorfstraße 12', 'Dorfstraße 13', 'Dorfstraße 14', 'Dorfstraße 17', 'Dorfstraße 18', 'Dorfstraße 19', 'Dorfstraße 2', 'Dorfstraße 20', 'Dorfstraße 21', 'Dorfstraße 23', 'Dorfstraße 27', 'Dorfstraße 28', 'Dorfstraße 29', 'Dorfstraße 3', 'Dorfstraße 30', 'Dorfstraße 31', 'Dorfstraße 32', 'Dorfstraße 33', 'Dorfstraße 34', 'Dorfstraße 35', 'Dorfstraße 36', 'Dorfstraße 37', 'Dorfstraße 38', 'Dorfstraße 39', 'Dorfstraße 4', 'Dorfstraße 40', 'Dorfstraße 41', 'Dorfstraße 5', 'Dorfstraße 6', 'Dorfstraße 7', 'Dorfstraße 8', 'Dorfstraße 9', 'Dorfstraße 9A', 'Gartenstraße 1', 'Gartenstraße 12', 'Gartenstraße 27', 'Gartenstraße 28', 'Grünstraße 25', 'Hanseatenweg 10', 'Hauptstraße 11', 'Hauptstraße 12', 'Hauptstraße 125', 'Hauptstraße 126', 'Hauptstraße 13', 'Hauptstraße 15', 'Hauptstraße 17', 'Hauptstraße 23', 'Hauptstraße 26', 'Hauptstraße 28', 'Hauptstraße 29', 'Hauptstraße 30', 'Hauptstraße 33', 'Hauptstraße 35', 'Hauptstraße 36', 'Hauptstraße 40', 'Hauptstraße 41', 'Hauptstraße 42', 'Hauptstraße 43', 'Hauptstraße 44', 'Hauptstraße 45', 'Hauptstraße 46', 'Hauptstraße 47', 'Hauptstraße 48', 'Hauptstraße 53', 'Hauptstraße 55', 'Hauptstraße 58', 'Hauptstraße 6', 'Inselstraße 5', 'Jägerstraße 1', 'Jägerstraße 2', 'Jägerstraße 3', 'Jägerstraße 4', 'Jägerstraße 5', 'Jägerstraße 9', 'Kastanienallee 26', 'Kirchstraße 1', 'Kirchstraße 2', 'Krumme Straße 4', 'Krumme Straße 6', 'Margaretenstraße 5', 'Mommsenstraße 5', 'Parkstraße 5', 'Rosenstraße 1', 'Rosenstraße 17', 'Scharnweberstraße 33', 'Scharnweberstraße 37', 'Scharnweberstraße 44', 'Scharnweberstraße 45', 'Scharnweberstraße 46', 'Scharnweberstraße 47', 'Scharnweberstraße 48', 'Scharnweberstraße 49', 'Sophienstraße 2', 'Sophienstraße 8']
#es reicht, wenn für die obigen Adressen auch nach Ortsteil gesucht wird, dann ist es auch eindeutig.
adressenBezirkOrtsteil={}
adressen=[]
dublAdressen=[]
keyAdresse=[]
bezKeyAdresse=[]
objAdr={}
objNrKeys=list(hidaDict.keys())
i=0
for n in range(0,len(objNrKeys)):
    key=objNrKeys[n]
    adresse=[]
    if 'Beziehung' in hidaDict[key].keys() and 'AdresseDict' in hidaDict[key].keys():
        key=objNrKeys[n]
        keyReduced=objNrKeys[n][:10]# da ',T,00...' keine Adresse oder Bezirk enthalten
        bezirk=hidaDict[keyReduced]['Bezirk']
        ortsteil=hidaDict[keyReduced]['Ortsteil']
        bezirkOrtsteil=bezirk+'-'+ortsteil
        bezList=list(set(list(hidaDict[key]['LinkGrad1ObjNr'])))
        for strasse in hidaDict[key]['AdresseDict'].keys():
            for hnr in hidaDict[key]['AdresseDict'][strasse]:
                elAdr=strasse+' '+hnr
                adrBezirkOrtsteil=elAdr+'-'+bezirk+'-'+ortsteil
                search=elAdr+' '+bezirk+' '+ortsteil
                adressenBezirkOrtsteil.setdefault(bezirkOrtsteil,[])
                #if (elAdr not in adressenBezirkOrtsteil[bezirkOrtsteil]):
                if (adrBezirkOrtsteil not in adressen):
                    adressenBezirkOrtsteil[bezirkOrtsteil].append(elAdr)
                    adressen.append(adrBezirkOrtsteil)
                    keyAdresse.append(key)
                else:
                    prevKey=keyAdresse[adressen.index(adrBezirkOrtsteil)]
                    bezBezKeyList=[]
                    bezBezKeyList.extend(bezList)
                    for elBezList in bezList:
                        bezBezKey=list(hidaDict[elBezList]['LinkGrad1ObjNr'])
                        bezBezKeyList.extend(bezBezKey)
                        #for elBezBezList in bezBezKey:
                        #    bezBezBezKey=list(hidaDict[elBezBezList]['LinkObjNr'])
                        #    bezbezKeyList.extend(bezBezBezKey)
                    bezbezKeyList=list(set(bezBezKeyList))
                    if not (prevKey in bezBezKeyList):
                        dublAdressen.append(search)
                        #print('i: ',i,'objNr: ',key,'bezList ',bezList,'bezbezKeyList: ',bezbezKeyList,'prevKey: ', prevKey,'elAdr: ', elAdr,'\n', 'bezirkOrtsteil: ', bezirkOrtsteil,)
                        i+=1
dublAdressen=sorted(list(set(dublAdressen)))
print(len(dublAdressen), dublAdressen)
#schauen bis zu welchem Grad Adressen weitergegeben werden

0 []
